In [1]:
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split 
from models import Implicit_ME
from models import Logistic
from models import SVM
from models import DecisionTree
from models import Explicit_ME
from collections import Counter
from imblearn.over_sampling import SMOTE

In [2]:
data = arff.loadarff('../dataSet/JIRA/wicket-1.3.0-beta2.arff')

In [3]:
df = pd.DataFrame(data[0])
df.head()

,CountDeclMethodPrivate,AvgLineCode,CountLine,MaxCyclomatic,CountDeclMethodDefault,AvgEssential,CountDeclClassVariable,SumCyclomaticStrict,AvgCyclomatic,AvgLine,...,DDEV,Added_lines,Del_lines,OWN_LINE,OWN_COMMIT,MINOR_COMMIT,MINOR_LINE,MAJOR_COMMIT,MAJOR_LINE,RealBugCount
0,0.0,10.0,212.0,8.0,0.0,1.0,0.0,18.0,2.0,18.0,...,3.0,396.0,5.0,0.966981,0.666667,0.0,1.0,3.0,1.0,0.0
1,0.0,4.0,273.0,3.0,0.0,1.0,1.0,18.0,1.0,12.0,...,2.0,546.0,0.0,0.937729,0.500000,0.0,2.0,2.0,0.0,0.0
2,0.0,4.0,231.0,2.0,0.0,1.0,1.0,12.0,1.0,14.0,...,3.0,463.0,1.0,0.917749,0.333333,0.0,2.0,3.0,1.0,0.0
3,0.0,6.0,104.0,2.0,0.0,1.0,1.0,4.0,1.0,16.0,...,2.0,210.0,0.0,0.971429,0.500000,0.0,1.0,2.0,1.0,0.0
4,0.0,5.0,128.0,2.0,0.0,1.0,1.0,5.0,1.0,18.0,...,2.0,258.0,0.0,0.968992,0.500000,0.0,1.0,2.0,1.0,0.0


In [4]:
df.describe()

,CountDeclMethodPrivate,AvgLineCode,CountLine,MaxCyclomatic,CountDeclMethodDefault,AvgEssential,CountDeclClassVariable,SumCyclomaticStrict,AvgCyclomatic,AvgLine,...,DDEV,Added_lines,Del_lines,OWN_LINE,OWN_COMMIT,MINOR_COMMIT,MINOR_LINE,MAJOR_COMMIT,MAJOR_LINE,RealBugCount
count,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,...,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000,1763.000000
mean,0.330119,6.966534,138.643222,2.413500,0.453205,1.063528,1.102666,10.542825,1.348837,12.187181,...,2.557572,280.017016,12.671583,0.681388,0.491734,0.019853,2.896767,2.537720,1.022689,0.110040
std,1.426574,6.736108,206.608787,2.990105,1.929943,0.563194,2.477872,22.169627,1.074539,8.960382,...,1.143234,444.672130,58.981782,0.198746,0.176911,0.256919,1.174650,1.073423,1.212190,0.475657
min,0.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,25.000000,0.000000,0.228070,0.153846,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,4.000000,46.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,7.000000,...,2.000000,86.000000,0.000000,0.535735,0.333333,0.000000,2.000000,2.000000,0.000000,0.000000
50%,0.000000,5.000000,79.000000,1.000000,0.000000,1.000000,1.000000,4.000000,1.000000,10.000000,...,2.000000,149.000000,0.000000,0.684211,0.500000,0.000000,3.000000,2.000000,1.000000,0.000000
75%,0.000000,9.000000,146.000000,3.000000,0.000000,1.000000,1.000000,10.000000,1.000000,14.000000,...,3.000000,288.000000,3.000000,0.835943,0.500000,0.000000,4.000000,3.000000,2.000000,0.000000
max,35.000000,112.000000,3650.000000,33.000000,27.000000,17.000000,54.000000,361.000000,26.000000,133.000000,...,10.000000,8133.000000,1022.000000,1.000000,1.000000,5.000000,6.000000,8.000000,8.000000,8.000000


In [5]:
X= df.iloc[ : , :-1].values
y= df['RealBugCount'].apply(lambda x : 1 if(x > 0) else 0)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
counterBefore = Counter(y_train)
counterBefore

Counter({1: 94, 0: 1316})

In [8]:
oversample = SMOTE()
Xbalance, ybalance = oversample.fit_resample(X_train, y_train)

In [9]:
counterAfter= Counter(ybalance)
counterAfter

Counter({1: 1316, 0: 1316})

In [10]:
# initial_centers = kmeans_plusplus_initializer(Xbalance, 1).initialize();
# xmeans_instance = xmeans(Xbalance, initial_centers);
# xmeans_instance.process();
# clusters = xmeans_instance.get_clusters();
# len(clusters)

In [11]:

# #separating data on the basis of clusters - explicit partioning
# clustX = []
# clusty = []

# arrX= np.array(Xbalance)
# arrY= np.array(ybalance)
# for i in range(len(clusters)):
#     tempx = []
#     tempy = []
#     for j in range(len(clusters[i])):
#         tempx.append(arrX[clusters[i][j]])
#         tempy.append(arrY[clusters[i][j]])
#     clustX.append(tempx)
#     clusty.append(tempy)
    

In [12]:
# for i in range(len(clusty)):
#     print(clusty[i])

In [13]:
DecisionTree(X_train,y_train,X_test,y_test)

Accuracy:  0.8781869688385269
F1 score:  0.6142076502732241
Auc score:  0.599763406940063


In [14]:
SVM(X_train,y_train,X_test,y_test)

Accuracy:  0.8980169971671388
F1 score:  0.47313432835820896
Auc score:  0.5


In [15]:
Logistic(X_train,y_train,X_test,y_test)

Accuracy:  0.9008498583569405
F1 score:  0.5666280823599565
Auc score:  0.5508236943568173


C:\Users\AdityaShankar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
Implicit_ME(X_train,y_train,X_test,y_test,10)

Accuracy of Mixture of Experts: 0.9065155807365439
F1 score 0.5913921919393875
AUC score 0.5662898703119523


In [17]:
Explicit_ME(X_train,y_train,X_test,y_test)

Accuracy of Mixture of Experts: 0.9036827195467422
F1 score 0.569635685599541
AUC score 0.5524009814230635


In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train,y_train) 
predictions = classifier.predict(X_test)
print("Accuracy of Mixture of Experts:",metrics.accuracy_score(y_test, predictions))
print("F1 score", f1_score(y_test,predictions, average='macro'))
arr2= confusion_matrix(y_test, predictions)
sn.heatmap(arr2, annot=True)


Accuracy of Mixture of Experts: 0.8923512747875354
F1 score 0.5910365853658537


NameError: name 'sn' is not defined